In [1]:
!pip3 install pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.7/777.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 17.7 MB/s eta 0:00:00


In [11]:
import numpy as np
import glob
import pytorch_lightning as pl
import joblib
import torch

from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from scipy.stats import ttest_1samp

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/experiment_design2/ex2/kakao-recoteam-recsys-2022-challenge/mlp

Mounted at /content/drive
/content/drive/MyDrive/experiment_design2/ex2/kakao-recoteam-recsys-2022-challenge/mlp


In [4]:
def validate_models(models, val_ds, use_filter=True, skip_val_only=False, r=None):
    hr = []
    for model in models:
        model = model.eval()
    s = 0
    tot_mrr = []
    ii = 0
    pbar = tqdm(DataLoader(val_ds, batch_size=250, shuffle=False))
    for input in pbar:
        (target_idx, _), (ret_mat, ret_cat, ret_scalar) = input
        for k in ret_mat:
            ret_mat[k] = ret_mat[k].cuda()
        for k in ret_cat:
            ret_cat[k] = ret_cat[k].cuda()
        ret_scalar = ret_scalar.cuda()
        rret = []
        for model in models:
            ret = model.forward(ret_mat, ret_cat, ret_scalar,)
            rret.append(ret)
        ret = torch.stack(rret)
        ret = torch.mean(ret, 0)
#         if use_filter:
#             ret[:, ~rec_cand_mask] = -1000000.0
        ret[ret_mat['item_bow'].bool()] = -10000.0
        if r is not None:
            print(r.shape)
            ret[:, ~r] = -100000.0
        top_rec = (-ret).argsort(-1)[:, :100].detach().cpu()

#         if skip_val_only:
#         top_rec = top_rec[target_idx != 29999]
#         target_idx = target_idx[target_idx != 29999]

        mrr = (top_rec == target_idx.unsqueeze(1)).float().numpy()
        hitst = mrr[:, :5].sum(-1)
        mrr = (mrr / np.expand_dims(np.arange(1, 1 + 100), 0)).sum(-1)
        tot_mrr.extend(mrr.tolist())
        hr.extend(hitst)
        pbar.set_postfix({
            'mrr': "%0.4f" % np.mean(tot_mrr),
            'hr': "%0.4f" % np.mean(hr)

                         })
    for model in models:
        model = model.train()
    return tot_mrr, hr

In [5]:
fname = "procseed_data_with_aug"
te_ds = joblib.load('%s/te_ds' % fname)

In [12]:
mrrs = []

for model_name in glob.glob("val_no_shuffle_mlp_256_*"):
  model = torch.load(model_name)
  mrr, hr = validate_models(model, te_ds)
  mrrs.append(np.mean(mrr))



100%|██████████| 164/164 [00:47<00:00,  3.42it/s, mrr=0.1903, hr=0.2672]


In [13]:
reported_mrr = 0.1913

t_statistic, p_value = ttest_1samp(mrrs, reported_mrr)

In [14]:
p_value

8.700303837668257e-07